In [245]:
import pandas as pd
import numpy as np 
import csv
import re
from difflib import SequenceMatcher

### **Problem 01:**

In [246]:
file_input = '../Data/temp_data.csv'
file_output = '../Data/clean_data.csv'

In [247]:
# Hàm sửa lỗi dư hoặc thiếu dấu ngoặc kép trong file CSV
def fix_csv_quotes(file_input, file_output):
    # Mở file đầu vào để đọc và file đầu ra để ghi
    with open(file_input, 'r', encoding = 'utf-8') as infile, open(file_output, 'w', newline = '', encoding = 'utf-8') as outfile:
        # Đọc tất cả các dòng từ file đầu vào
        reader = infile.readlines()
        # Tạo writer để ghi vào file đầu ra
        writer = csv.writer(outfile)

        # Lặp qua từng dòng trong file đầu vào
        for line in reader:
            # Kiểm tra số lượng dấu ngoặc kép trong dòng
            num_quotes = line.count('"')
            # Nếu số lượng dấu ngoặc kép là số lẻ, thêm một dấu ngoặc kép ở cuối dòng
            if num_quotes % 2 != 0:
                line = line.strip() + '"\n'
            # Ghi lại dòng đã sửa vào file đầu ra
            writer.writerow(next(csv.reader([line], skipinitialspace = True)))

In [248]:
fix_csv_quotes(file_input, file_output)

### **Problem 02:**

In [249]:
column_index = 8  # Chỉ số của cột cần làm sạch (bắt đầu từ 0), cột 8 là cột Description

def clean_string(string):

    """
    Hàm này sẽ làm sạch chuỗi bằng cách loại bỏ các biểu tượng và ký tự không cần thiết.
    (nhưng giữ lại các kí tự chữ (hoa và thường), số, dấu chấm, dấu phẩy, dấu | )
    
    string: Chuỗi cần làm sạch.
    cleanString: Chuỗi đã được làm sạch.

    """

    # Hàm isinstance check xem data input của mình có phải là string không 
    if isinstance(string, str):
        cleanString = re.sub(r'[^\w\s.,|]', '', string) # Xoá tất cả các kí tự ngoại trừ chữ (thường, hoa), số, dấu chấm, dấu phẩy, dấu |
        return cleanString
    else:
        return string

In [250]:
def clean_file_csv(file_input, file_output, column_index):

    """
    Hàm này sẽ làm sạch một cột cụ thể trong file CSV, do có những cột khác (VD cột Postdate có dấu : ), 
    Vì vậy nếu apply full file csv nó sẽ fix hết cả những cột không cần thiết -> chỉ áp dụng chính xác cột cần xử lí thôi.

    input_csv (str): Đường dẫn tới file CSV đầu vào (temp_data).
    output_csv (str): Đường dẫn tới file CSV đầu ra (clean_data).
    column_index (int): Chỉ số của cột cần làm sạch.
    """

    # Đọc file CSV vào DataFrame
    df = pd.read_csv(file_input, on_bad_lines = 'skip')

    # Kiểm tra xem chỉ số cột có hợp lệ không
    if column_index < len(df.columns):
        # Áp dụng hàm làm sạch cho cột cụ thể
        col = df.columns[column_index]
        df[col] = df[col].apply(clean_string)
    else:
        print(f"Invalid column index !")

    # Lưu lại DataFrame đã được làm sạch thành file CSV mới
    df.to_csv(file_output, index = False)



In [251]:
clean_file_csv(file_input, file_output, column_index)

### **Problem 03:**

In [252]:
# Đọc lại file csv, xoá các dòng bị lỗi 
# (mình nhận thấy số lượng dòng bị lỗi không nhiều ~ 10 dòng nên mình sẽ skip luôn vì tụi mình có nhiều data mà =))))
df = pd.read_csv('../Data/clean_data.csv', on_bad_lines = 'skip')

In [253]:
print(df)

         No Price                                              Title   Area  \
0         1    TT  Cửa gỗ công nghiệp MDF laminate cao cấp tại Qu...    NaN   
1         2    TT   ��The power of online advertising facebook ads��    NaN   
2         3    TT  ��️ Email/SMS Marketing - Sustainable communic...    NaN   
3         4  13.5  Siêu rẻ ! Nhà 1 Đời Chủ, 2 MT Hoàng Sa, Q1, Đo...   44.0   
4         5    TT  Maxads - leading the trend of MULTIMEDIA ADVER...    NaN   
...     ...   ...                                                ...    ...   
9979  10046     7  BÁN NHÀ Ở NGAY TÔ NGỌC VÂN, 170M2 , LINH ĐÔNG ...  170.0   
9980  10047   3.3  Bán Nhà Mặt Tiền Quốc lộ 1A, Bình Chiểu, Thủ Đ...   40.0   
9981  10048    13  Bán nhà MT Kinh doanh đường Đặng Văn Bi, P. Bì...   52.8   
9982  10049    TT  Nhà 1 tầng, diện tích đất 112m², đường số 9, T...  112.0   
9983  10050     6  BÁN NHÀ MỚI ĐẸP, HXH QUAY ĐẦU, TẶNG FUUL NỘI T...  100.0   

      Bedrooms  WCs District             Postdate  

In [254]:
# Thay thế "Thoả thuận" bằng NaN cho dễ xử lí về sau (tạm thời)
df['Price'] = df['Price'].replace('TT', np.nan)

In [255]:
# Các biểu thức chính quy để trích xuất thông tin
price_pattern = re.compile(r'Giá[^0-9]*([0-9]+[.,]?[0-9]*)\s*(Tỷ)', re.IGNORECASE)
area_pattern = re.compile(r'(\d+\.?\d*)\s*x\s*(\d+\.?\d*)|\((\d+\.?\d*)m2\)')
bedroom_pattern = re.compile(r'(\d+)\s*PN')
wc_pattern = re.compile(r'(\d+)\s*WC')

In [256]:
# Hàm để trích xuất thông tin từ một chuỗi văn bản
def extract_info(text):
    if text is None or pd.isna(text):
        text = ''  #fix: Khởi tạo giá trị rỗng nếu text là None hoặc NaN
    elif not isinstance(text, str):
        text = str(text) #fix: Chuyển đổi giá trị sang chuỗi nếu không phải là chuỗi
    
    price_match = price_pattern.search(text)
    area_match = area_pattern.search(text)
    bedroom_match = bedroom_pattern.search(text)
    wc_match = wc_pattern.search(text)
    
    price = None
    if price_match:
        number = price_match.group(1).replace(',', '.')
        unit = price_match.group(2)
        if unit and unit.lower() in ['tỷ', 'tỷ', 'tỉ']:
            price = float(number) * 1_000_000_000  # Chuyển đổi tỷ sang VND
        elif unit and unit.lower() in ['triệu', 'triệu']:
            price = float(number) * 1_000_000  # Chuyển đổi triệu sang VND
        else:
            price = float(number)  # Nếu không có đơn vị, chỉ trả về số dưới dạng float
    
    area = None
    if area_match:
        if area_match.group(3):
            area = float(area_match.group(3))  # Diện tích tổng (150m2)
        else:
            width = float(area_match.group(1))
            length = float(area_match.group(2))
            area = width * length  # Tính diện tích từ chiều dài và chiều rộng
    
    bedrooms = bedroom_match.group(1) if bedroom_match else None
    wcs = wc_match.group(1) if wc_match else None
    
    return pd.Series([price, area, bedrooms, wcs])

In [257]:
# Hàm để trích xuất thông tin từ cả 'Title' và 'Description'
def extract_info_from_row(row):
    title_info = extract_info(row['Title'])
    description_info = extract_info(row['Description'])
    
    # Nếu thông tin từ Description không có, lấy thông tin từ Title
    final_info = title_info.combine_first(description_info)
    
    return final_info

In [258]:
df[['Extracted_Price', 'Extracted_Area', 'Extracted_Bedrooms', 'Extracted_WCs']] = df.apply(extract_info_from_row, axis=1) #fix

df['Price'] = pd.to_numeric(df['Price'], errors='coerce')


# Cập nhật cột Price theo điều kiện
df['Price'] = df.apply(lambda row: row['Extracted_Price'] if pd.isnull(row['Price']) or row['Price'] > 1000 else row['Price'], axis=1)

# Hàm để điền thông tin bedrooms và wcs từ các giá trị đã trích xuất
def fill_bedrooms_and_wcs(row):
    if pd.isnull(row['Bedrooms']):
        row['Bedrooms'] = row['Extracted_Bedrooms']
    if pd.isnull(row['WCs']):
        row['WCs'] = row['Extracted_WCs']
    return row

# Áp dụng hàm để điền thông tin
df = df.apply(fill_bedrooms_and_wcs, axis=1)

# Xuất DataFrame mới ra file CSV
df.to_csv(file_output, index=False)

print(df.head())

C:\Users\Admin\AppData\Local\Temp\ipykernel_10624\1363349392.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  final_info = title_info.combine_first(description_info)


   No  Price                                              Title  Area  \
0   1    NaN  Cửa gỗ công nghiệp MDF laminate cao cấp tại Qu...   NaN   
1   2    NaN   ��The power of online advertising facebook ads��   NaN   
2   3    NaN  ��️ Email/SMS Marketing - Sustainable communic...   NaN   
3   4   13.5  Siêu rẻ ! Nhà 1 Đời Chủ, 2 MT Hoàng Sa, Q1, Đo...  44.0   
4   5    NaN  Maxads - leading the trend of MULTIMEDIA ADVER...   NaN   

  Bedrooms   WCs District             Postdate  \
0      NaN   NaN   quan-1  2023-11-22 11:08:20   
1     None  None   quan-1  2023-11-23 10:27:56   
2     None  None   quan-1  2023-11-24 09:44:58   
3      NaN   NaN   quan-1  2023-11-24 16:39:03   
4     None  None   quan-1  2023-11-25 09:55:56   

                                         Description  Extracted_Price  \
0   |HOTLINE  zalo 035.312.6411  0888.696.029 NVK...              NaN   
1  advertising forms quotas a springboardquot bec...              NaN   
2  interaction strategy. Connect and buil

In [259]:
print(df)

         No         Price                                              Title  \
0         1           NaN  Cửa gỗ công nghiệp MDF laminate cao cấp tại Qu...   
1         2           NaN   ��The power of online advertising facebook ads��   
2         3           NaN  ��️ Email/SMS Marketing - Sustainable communic...   
3         4  1.350000e+01  Siêu rẻ ! Nhà 1 Đời Chủ, 2 MT Hoàng Sa, Q1, Đo...   
4         5           NaN  Maxads - leading the trend of MULTIMEDIA ADVER...   
...     ...           ...                                                ...   
9979  10046  7.000000e+00  BÁN NHÀ Ở NGAY TÔ NGỌC VÂN, 170M2 , LINH ĐÔNG ...   
9980  10047  3.300000e+00  Bán Nhà Mặt Tiền Quốc lộ 1A, Bình Chiểu, Thủ Đ...   
9981  10048  1.300000e+01  Bán nhà MT Kinh doanh đường Đặng Văn Bi, P. Bì...   
9982  10049  5.600000e+09  Nhà 1 tầng, diện tích đất 112m², đường số 9, T...   
9983  10050  6.000000e+00  BÁN NHÀ MỚI ĐẸP, HXH QUAY ĐẦU, TẶNG FUUL NỘI T...   

       Area Bedrooms   WCs District    

In [260]:
# Loại bỏ cột No và cột Title không cần thiết 
df = df.drop(columns = df.columns[[0, 2]])



# # Thiết lập ngưỡng cho số lượng giá trị NaN tối đa cho phép trong một hàng
# max_nan = 3  

# # Xóa các hàng có số lượng giá trị NaN lớn hơn ngưỡng
# df = df.dropna(thresh = len(df.columns) - max_nan)  # thresh: chỉ định số lượng giá trị NOT NULL cần thiết để giữ lại hàng

In [261]:
print(df)

             Price   Area Bedrooms   WCs District             Postdate  \
0              NaN    NaN      NaN   NaN   quan-1  2023-11-22 11:08:20   
1              NaN    NaN      NaN   NaN   quan-1  2023-11-23 10:27:56   
2              NaN    NaN      NaN   NaN   quan-1  2023-11-24 09:44:58   
3     1.350000e+01   44.0      NaN   NaN   quan-1  2023-11-24 16:39:03   
4              NaN    NaN      NaN   NaN   quan-1  2023-11-25 09:55:56   
...            ...    ...      ...   ...      ...                  ...   
9979  7.000000e+00  170.0     None  None  thu-duc  2022-10-27 16:24:29   
9980  3.300000e+00   40.0      4.0   2.0  thu-duc  2022-10-27 14:48:46   
9981  1.300000e+01   52.8      4.0   5.0  thu-duc  2022-10-27 09:36:50   
9982  5.600000e+09  112.0      3.0   2.0  thu-duc  2022-10-26 22:29:32   
9983  6.000000e+00  100.0     None  None  thu-duc  2022-10-25 13:03:53   

                                            Description  Extracted_Price  \
0      |HOTLINE  zalo 035.312.6411 

In [262]:
# Chuyển đổi các cột 'Price' và 'Area' sang kiểu số (float)
df['Price'] = df['Price'].astype(float)
df['Area'] = df['Area'].astype(float)

In [263]:
# Hàm xử lý cột Price
def process_price(df):
    def convert_price(price):
        if pd.isnull(price):
            return price
        price = float(price)
        if price > 1e9:
            return price / 1e9  # Nếu giá trị lớn hơn 1 tỷ, chuyển sang đơn vị tỷ
        elif price > 1e6:
            return price / 1e6  # Nếu giá trị lớn hơn 1 triệu, chuyển sang đơn vị triệu
        elif price > 1e3:
            return price / 1e3  # Nếu giá trị lớn hơn 1 nghìn, chuyển sang đơn vị nghìn
        return price  # Trả về giá trị gốc nếu nhỏ hơn 1 nghìn
    
    df['Price'] = df['Price'].apply(convert_price)  # Áp dụng hàm convert_price cho cột 'Price'
    return df

df = process_price(df)

# Hàm tính giá trên một mét vuông
def price_per_sqm(df):
    df['Price_per_sqm'] = df.apply(lambda row: row['Price'] / row['Area'] if pd.notnull(row['Price']) and pd.notnull(row['Area']) else np.nan, axis=1)
    # Tạo cột 'Price_per_sqm' bằng cách lấy 'Price' chia cho 'Area' nếu cả hai đều không bị null, ngược lại trả về NaN
    return df




# Hàm điền giá trị thiếu
def fill_miss_vals(df):
    # Tính giá trị trung bình của 'Price', 'Area', và 'Price_per_sqm' theo 'District'
    district_means = df.groupby('District').mean(numeric_only=True)
    
    # Điền giá trị thiếu cho 'Price' bằng giá trị trung bình của 'District'
    for idx, row in df.iterrows():
        if pd.isnull(row['Price']):
            df.at[idx, 'Price'] = district_means.loc[row['District'], 'Price']
    
    # Tính lại giá trên một mét vuông
    df = price_per_sqm(df)
    
    # Điền giá trị thiếu cho 'Area' và 'Price' dựa trên 'Price_per_sqm'
    for idx, row in df.iterrows():
        if pd.isnull(row['Area']) and pd.notnull(row['Price']):
            df.at[idx, 'Area'] = row['Price'] / row['Price_per_sqm']  # Tính diện tích dựa trên giá trị 'Price' và 'Price_per_sqm'
        elif pd.isnull(row['Price']) and pd.notnull(row['Area']):
            df.at[idx, 'Price'] = row['Area'] * row['Price_per_sqm']  # Tính giá trị 'Price' dựa trên 'Area' và 'Price_per_sqm'
        elif pd.isnull(row['Price']) and pd.isnull(row['Area']):
            df.at[idx, 'Price'] = district_means.loc[row['District'], 'Price']  # Điền 'Price' bằng giá trị trung bình của quận
            df.at[idx, 'Area'] = district_means.loc[row['District'], 'Area']  # Điền 'Area' bằng giá trị trung bình của quận
    
    
    # Xử lý các giá trị 'Area' vẫn còn thiếu
    for idx, row in df.iterrows():
        if pd.isnull(row['Area']):  
            df.at[idx, 'Area'] = district_means.loc[row['District'], 'Area']

    # Điền giá trị thiếu cho 'Price_per_sqm' nếu còn thiếu
    for idx, row in df.iterrows():
        if pd.isnull(row['Price_per_sqm']) and pd.notnull(row['Price']) and pd.notnull(row['Area']):
            df.at[idx, 'Price_per_sqm'] = row['Price'] / row['Area']  # Tính lại 'Price_per_sqm' nếu thiếu và các giá trị khác không bị null
    

    # Làm tròn các giá trị vừa điền vào
    df['Price'] = df['Price'].round(2)  # Làm tròn giá trị 'Price' đến 2 chữ số thập phân
    df['Area'] = df['Area'].round(2)  # Làm tròn giá trị 'Area' đến 2 chữ số thập phân
    df['Price_per_sqm'] = df['Price_per_sqm'].round(2)  # Làm tròn giá trị 'Price_per_sqm' đến 2 chữ số thập phân

    return df





In [264]:
# Kiểm tra giá trị null trong cột 'Area' và 'Price'
null_area = df['Area'].isnull().sum()
print(null_area)
null_price = df['Price'].isnull().sum()
print(null_price)

1012
817


In [265]:
# Thêm cột giá trên 1 mét vuông
df = price_per_sqm(df)

In [266]:
# Hàm để cập nhật cột Price theo điều kiện price_per_sqm > 1
def update_price(row):
    if row['Area'] and row['Area'] > 0: #fix: Đảm bảo diện tích không phải là None hoặc 0
        price_per_sqm = row['Price'] / row['Area']
        if price_per_sqm > 1:
            return row['Extracted_Price']
    return row['Price']

# Áp dụng hàm update_price
df['Price'] = df.apply(update_price, axis=1)

In [267]:
# Điền giá trị thiếu cho 'Price' và 'Area'
df = fill_miss_vals(df)

In [268]:
df = process_price(df)

In [269]:
print(df)

          Price    Area Bedrooms   WCs District             Postdate  \
0      1.769412  151.69      NaN   NaN   quan-1  2023-11-22 11:08:20   
1      1.769412  151.69      NaN   NaN   quan-1  2023-11-23 10:27:56   
2      1.769412  151.69      NaN   NaN   quan-1  2023-11-24 09:44:58   
3     13.500000   44.00      NaN   NaN   quan-1  2023-11-24 16:39:03   
4      1.769412  151.69      NaN   NaN   quan-1  2023-11-25 09:55:56   
...         ...     ...      ...   ...      ...                  ...   
9979   7.000000  170.00     None  None  thu-duc  2022-10-27 16:24:29   
9980   3.300000   40.00      4.0   2.0  thu-duc  2022-10-27 14:48:46   
9981  13.000000   52.80      4.0   5.0  thu-duc  2022-10-27 09:36:50   
9982   5.600000  112.00      3.0   2.0  thu-duc  2022-10-26 22:29:32   
9983   6.000000  100.00     None  None  thu-duc  2022-10-25 13:03:53   

                                            Description  Extracted_Price  \
0      |HOTLINE  zalo 035.312.6411  0888.696.029 NVK...    

In [270]:
# Xử lí cột Postdate (chỉ giữ lại Ngày xoá Giờ)
# Chuyển đổi cột 'Postdate' sang kiểu datetime
df['Postdate'] = pd.to_datetime(df['Postdate'])

# Tạo một cột mới chỉ chứa ngày
df['Date'] = df['Postdate'].dt.date

# Tìm vị trí (index) của cột 'Postdate'
col_index = df.columns.get_loc('Postdate')

# Xóa cột 'Postdate' cũ 
df = df.drop(columns = ['Postdate'])

# Chèn cột 'Date' mới vào đúng vị trí cũ của cột 'Postdate'
df.insert(col_index, 'Postdate', df.pop('Date'))

In [271]:
# Load lại những thay đổi vào file clean_data.csv
df.to_csv(file_output, index = False)

In [272]:
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Extracted_Price,Extracted_Area,Extracted_Bedrooms,Extracted_WCs,Price_per_sqm
0,1.769412,151.69,NaN,NaN,quan-1,2023-11-22,|HOTLINE zalo 035.312.6411 0888.696.029 NVK...,NaN,4950.0,NaN,NaN,11664943.37
1,1.769412,151.69,None,None,quan-1,2023-11-23,advertising forms quotas a springboardquot bec...,NaN,NaN,None,None,11664943.37
2,1.769412,151.69,None,None,quan-1,2023-11-24,interaction strategy. Connect and build a clos...,NaN,NaN,None,None,11664943.37
3,13.500000,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",1.350000e+10,44.0,NaN,NaN,0.31
4,1.769412,151.69,None,None,quan-1,2023-11-25,Youtube Ads is a form of advertising that hel...,NaN,NaN,None,None,11664943.37
5,1.769412,151.69,None,None,quan-1,2023-11-25,customer trust. Is your business unable to pro...,NaN,NaN,None,None,11664943.37
6,1.769412,151.69,None,None,quan-1,2023-11-25,customer trust. Is your business unable to pro...,NaN,NaN,None,None,11664943.37
7,8.700000,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,None,None,0.19
8,8.700000,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,None,None,0.19
9,21.500000,64.00,None,None,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",NaN,NaN,None,None,0.34


In [273]:
print(df)

          Price    Area Bedrooms   WCs District    Postdate  \
0      1.769412  151.69      NaN   NaN   quan-1  2023-11-22   
1      1.769412  151.69      NaN   NaN   quan-1  2023-11-23   
2      1.769412  151.69      NaN   NaN   quan-1  2023-11-24   
3     13.500000   44.00      NaN   NaN   quan-1  2023-11-24   
4      1.769412  151.69      NaN   NaN   quan-1  2023-11-25   
...         ...     ...      ...   ...      ...         ...   
9979   7.000000  170.00     None  None  thu-duc  2022-10-27   
9980   3.300000   40.00      4.0   2.0  thu-duc  2022-10-27   
9981  13.000000   52.80      4.0   5.0  thu-duc  2022-10-27   
9982   5.600000  112.00      3.0   2.0  thu-duc  2022-10-26   
9983   6.000000  100.00     None  None  thu-duc  2022-10-25   

                                            Description  Extracted_Price  \
0      |HOTLINE  zalo 035.312.6411  0888.696.029 NVK...              NaN   
1     advertising forms quotas a springboardquot bec...              NaN   
2     interacti

In [274]:
def remove_unusual_line_terminators(file_input, file_output):
    with open(file_input, 'r', encoding='utf-8') as infile:
        content = infile.read()

    # Loại bỏ các ký tự kết thúc dòng không bình thường (LS và PS)
    cleaned_content = content.replace('\u2028', '\n').replace('\u2029', '\n')

    with open(file_output, 'w', encoding='utf-8', newline='') as outfile:
        outfile.write(cleaned_content)

In [275]:
def remove_duplicates(df):
    # Đảm bảo rằng các giá trị trong 'Postdate' và 'Description' là chuỗi
    df['Postdate'] = df['Postdate'].astype(str)
    df['Description'] = df['Description'].astype(str)
    
    # Bước 1: Loại bỏ các dòng trùng lặp mà tất cả các cột đều giống nhau
    df = df.drop_duplicates()
    
    # Bước 2: Nhóm các dòng theo các cột 'Price', 'Area', 'Bedrooms', 'WCs', 'District'
    grouped = df.groupby(['Price', 'Area', 'Bedrooms', 'WCs', 'District'], dropna=False)
    
    # Danh sách các chỉ số của các dòng cần giữ lại
    indices_to_keep = set(df.index)

    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    # Lặp qua từng nhóm để kiểm tra sự tương đồng
    for _, group in grouped:
        group_indices = list(group.index)
        for i in range(len(group_indices)):
            for j in range(i + 1, len(group_indices)):
                idx1, idx2 = group_indices[i], group_indices[j]
                row1, row2 = df.loc[idx1], df.loc[idx2]
                # Kiểm tra sự tương đồng của 'Description' và ngày 'Postdate'
                postdate_similarity = similar(row1['Postdate'], row2['Postdate'])
                description_similarity = similar(row1['Description'], row2['Description'])
                if postdate_similarity > 0.75 and description_similarity > 0.5:
                    indices_to_keep.discard(idx2)
    
    # Chuyển đổi tập hợp các chỉ số cần giữ lại thành danh sách
    indices_to_keep = list(indices_to_keep)
    
    # Lọc lại DataFrame với các chỉ số cần giữ lại
    df_cleaned = df.loc[indices_to_keep].reset_index(drop=True)
    
    # Trả về DataFrame đã loại bỏ các dòng trùng lặp
    return df_cleaned

In [276]:
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Extracted_Price,Extracted_Area,Extracted_Bedrooms,Extracted_WCs,Price_per_sqm
0,1.769412,151.69,NaN,NaN,quan-1,2023-11-22,|HOTLINE zalo 035.312.6411 0888.696.029 NVK...,NaN,4950.0,NaN,NaN,11664943.37
1,1.769412,151.69,None,None,quan-1,2023-11-23,advertising forms quotas a springboardquot bec...,NaN,NaN,None,None,11664943.37
2,1.769412,151.69,None,None,quan-1,2023-11-24,interaction strategy. Connect and build a clos...,NaN,NaN,None,None,11664943.37
3,13.500000,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",1.350000e+10,44.0,NaN,NaN,0.31
4,1.769412,151.69,None,None,quan-1,2023-11-25,Youtube Ads is a form of advertising that hel...,NaN,NaN,None,None,11664943.37
5,1.769412,151.69,None,None,quan-1,2023-11-25,customer trust. Is your business unable to pro...,NaN,NaN,None,None,11664943.37
6,1.769412,151.69,None,None,quan-1,2023-11-25,customer trust. Is your business unable to pro...,NaN,NaN,None,None,11664943.37
7,8.700000,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,None,None,0.19
8,8.700000,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,None,None,0.19
9,21.500000,64.00,None,None,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",NaN,NaN,None,None,0.34


In [277]:
# Áp dụng hàm remove_duplicates lên DataFrame đã được làm sạch
df = remove_duplicates(df)

In [278]:
# DataFrame sau khi thêm cột "No" và loại bỏ duplicate 
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Extracted_Price,Extracted_Area,Extracted_Bedrooms,Extracted_WCs,Price_per_sqm
0,1.769412,151.69,NaN,NaN,quan-1,2023-11-22,|HOTLINE zalo 035.312.6411 0888.696.029 NVK...,NaN,4950.0,NaN,NaN,11664943.37
1,1.769412,151.69,None,None,quan-1,2023-11-23,advertising forms quotas a springboardquot bec...,NaN,NaN,None,None,11664943.37
2,1.769412,151.69,None,None,quan-1,2023-11-24,interaction strategy. Connect and build a clos...,NaN,NaN,None,None,11664943.37
3,13.500000,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",1.350000e+10,44.0,NaN,NaN,0.31
4,1.769412,151.69,None,None,quan-1,2023-11-25,Youtube Ads is a form of advertising that hel...,NaN,NaN,None,None,11664943.37
5,1.769412,151.69,None,None,quan-1,2023-11-25,customer trust. Is your business unable to pro...,NaN,NaN,None,None,11664943.37
6,8.700000,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,None,None,0.19
7,21.500000,64.00,None,None,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",NaN,NaN,None,None,0.34
8,9.300000,40.00,4.0,4.0,quan-1,2023-11-29,"Kết cấu 1 trệt, 2 lầu, sân thượng, 3 PN, 4WC....",NaN,NaN,3,4,0.23
9,130.000000,455.00,NaN,NaN,quan-1,2023-10-25,Diện tích 13 x 35m. | Hiện trạng Nhà cấp 4. |...,1.300000e+11,455.0,NaN,NaN,0.29


In [279]:
# Xem kiểu dữ liệu của cột "Postdate"
df['Postdate'].dtype

dtype('O')

In [280]:
# Chuyển kiểu dữ liệu của cột "Postdate" sang dạng datetime và check lại kiểu dữ liệu 1 lần nữa
df['Postdate'] = pd.to_datetime(df['Postdate'])
df['Postdate'].dtype

dtype('<M8[ns]')

In [281]:
# Loại bỏ các data có ngày đăng tin là 2022 và lưu lại vào file csv
df = df[df['Postdate'].dt.year != 2022]

In [282]:
def remove_high_price_rows(df):
    return df[df['Price_per_sqm'] <= 2]

df = remove_high_price_rows(df)

In [283]:
def remove_invalid(df):
    return df[df['Price_per_sqm'] >= 0.03]

df = remove_invalid(df)

In [284]:
# Danh sách các quận cần chia giá trị Price cho 10^3
districts = ['binh-chanh', 'binh-tan', 'hoc-mon', 'cu-chi', 'tan-binh', 'quan-8', 'quan-12']

def adjust_price_for_district(row):
    if any(district in row['District'] for district in districts) and row['Price_per_sqm'] > 1:
        return row['Price'] / 1_000
    return row['Price']

# Áp dụng hàm adjust_price_for_district
df['Price'] = df.apply(adjust_price_for_district, axis=1)


In [285]:
# Tạo lại cột No
df.insert(0, 'No', range(1, len(df) + 1))

In [286]:
# Lưu lại DataFrame vào file CSV
df.to_csv(file_output, index = False)

In [287]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()
missing_values

No                       0
Price                    0
Area                     0
Bedrooms              1834
WCs                   2182
District                 0
Postdate                 0
Description              0
Extracted_Price       3535
Extracted_Area        4039
Extracted_Bedrooms    4422
Extracted_WCs         4638
Price_per_sqm            0
dtype: int64